In [ ]:
# Neural Nets imports
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
from sklearn import metrics

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
#### CNN Model ##########
def format_to_cnn(df):
    X = np.expand_dims(df, axis=2)
    return X

In [ ]:
verboseLevel=0
validationSplit=0.2
batchSize=30
epochs=1000

In [ ]:
# callback preparation
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)

In [ ]:
def createReduceRLObject():
    reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)
    return reduce_lr

In [ ]:
inputSize = 9
colList = ['HiddenLayers', 'R2Score', 'MAE', 'MSE', 'H5FileName', 'TrainHistory', 'TrainPredictions']

In [ ]:
#Define CNN model
    
def cnn_model_mse(inputSize=inputSize):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=6, activation=custom_activation,
                     #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                     #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                     #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                     input_shape=(inputSize,1)))
    model.add(Conv1D(filters=64, kernel_size=3, 
                     #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                     #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                     #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                     activation=custom_activation))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                    activation=custom_activation))
    #model.add(BatchNormalization())
    model.add(Dense(1, 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    #kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),                   
                    activation="linear"))

    optmzr=Adam(lr=0.001)
    model.compile(optimizer=optmzr, loss='mse', metrics=['mse'])

    return model


In [ ]:
# Transform the datasets for LSTM processing
cnn_X_test1_summaryL1Z2_std = format_to_cnn(X_test1_summaryL1Z2_std)
cnn_X_test1_summaryL1Z2_minMax = format_to_cnn(X_test1_summaryL1Z2_minMax)
cnn_X_train_base_std = format_to_cnn(X_train_base_std)
cnn_X_train_base_minMax = format_to_cnn(X_train_base_minMax)
cnn_X_train_all_std = format_to_cnn(X_train_all_std)
cnn_X_train_all_minMax = format_to_cnn(X_train_all_minMax)
cnn_X_test_all_std = format_to_cnn(X_test_all_std)
cnn_X_test_all_minMax = format_to_cnn(X_test_all_minMax)

In [ ]:
#MinMax All MSE

In [ ]:
cnn_regr_minMax_all_mse = cnn_model_mse()

In [ ]:
checkpoint_cnn_minMax_all_mse = ModelCheckpoint("cnn_model_minMax_all_mse.h5",
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
reduce_lr_minMax_all_mse = createReduceRLObject()
callbacks_list_cnn_minMax_all_mse = [checkpoint_cnn_minMax_all_mse, reduce_lr_minMax_all_mse]

In [ ]:
%%time
history_CNN_minMax_all_mse = cnn_regr_minMax_all_mse.fit(cnn_X_train_all_minMax, y_train_scaled_minMax_all,
                            batch_size=30, 
                            validation_split=0.2, 
                            epochs=epochs, verbose=verboseLevel,
                            callbacks=callbacks_list_cnn_minMax_all_mse)

In [ ]:
cnn_regr_minMax_all_mse_new =load_model('cnn_model_minMax_all_mse.h5')
#Predict
y_pred_CNN_minMax_all_mse_scaled = cnn_regr_minMax_all_mse_new.predict(cnn_X_test_all_minMax)
#reverse scaling to compare to true values
pred = targetMinMaxScalerAll.inverse_transform(y_pred_CNN_minMax_all_mse_scaled)

In [ ]:
#if metrics.mean_absolute_error(y_test_all_minMax, pred) < minMaeAWS:
minMseAWS = metrics.mean_squared_error(y_test_all_minMax, pred)
minRmseAWS = np.sqrt(minMseAWS)
minMaeAWS = metrics.mean_absolute_error(y_test_all_minMax, pred)
minR2AWS = metrics.r2_score(y_test_all_minMax, pred)
modelNameAWS = "CNN_MinMax_ALL_MSE"

In [ ]:
delta = pd.DataFrame()
delta = (1-(pd.DataFrame(pred)  / y_test_all_minMax.to_numpy()))*100
delta = delta.rename(columns={delta.columns[0]: "ThroughputDeltaDeviationPercentage"})

In [ ]:
mpd_all = delta.median()[0]

In [ ]:
def mape(test, pred):
    x = 0
    for i in range(len(test)):
        x+=np.abs((test[test.columns[0]][i] - pred[i])/test[test.columns[0]][i])
    return (x/len(test)*100)

In [ ]:
mape_all = mape(y_test_all_minMax, pred)

In [ ]:
minMaeAWS, minR2AWS, modelNameAWS, minMseAWS, minRmseAWS